In [5]:
from huggingface_hub import notebook_login

notebook_login()


In [1]:
from modeling_esm_plusplus import ESMplusplusForMaskedLM

In [4]:
model = ESMplusplusForMaskedLM.from_pretrained_esm('esmc_600m')
model

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

ESMplusplusForMaskedLM(
  (embed): Embedding(64, 1152)
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0-35): 36 x UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1152, out_features=3456, bias=False)
          )
          (out_proj): Linear(in_features=1152, out_features=1152, bias=False)
          (q_ln): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (k_ln): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (rotary): RotaryEmbedding()
        )
        (ffn): Sequential(
          (0): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=1152, out_features=6144, bias=False)
          (2): SwiGLU()
          (3): Linear(in_features=3072, out_features=1152, bias=False)
        )
      )
    )
    (norm): LayerNorm((1152,), eps=1e-05, element

In [6]:
model.tokenizer

EsmSequenceTokenizer(name_or_path='', vocab_size=33, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<cls>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['|']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	31: AddedToken("|", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False,

In [5]:
model.push_to_hub('Synthyra/ESMplusplus_large', private=True)

model.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Synthyra/ESMplusplus_large/commit/d3fe8183064acd0b83a10b7f5d815694a5954e29', commit_message='Upload ESMplusplusForMaskedLM', commit_description='', oid='d3fe8183064acd0b83a10b7f5d815694a5954e29', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
import torch

In [5]:
torch.rand(1, 32, 64) * torch.randint(0, 2, (1, 32)).unsqueeze(-1)


tensor([[[0.9743, 0.5183, 0.2230,  ..., 0.2277, 0.8020, 0.4080],
         [0.2482, 0.0317, 0.3162,  ..., 0.8049, 0.3936, 0.0551],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0940, 0.7385, 0.1063,  ..., 0.6314, 0.5629, 0.8297]]])